In [24]:
! pip install faiss-cpu
! pip install pandas
! pip install mistralai
! pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

In [13]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("Type your API key")
client = Mistral(api_key=api_key)

Type your API key ········


In [14]:
def extract_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    # Extract text from the HTML content
    text = soup.get_text(separator=' ', strip=True)
    return text

In [15]:
class WebCrawler:
    
    def __init__(self, base_url):
        self.base_url = base_url
        self.visited_urls = set()
        self.content = dict()
        self.crawl_limit = 0

    def is_valid_url(self, url):
        # Check if the URL is within the same domain
        parsed_url = urlparse(url)
        return bool(parsed_url.netloc) and parsed_url.netloc == urlparse(self.base_url).netloc

    def fetch_page(self, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            return None

    def parse_links(self, html, base_url):
        soup = BeautifulSoup(html, 'html.parser')
        links = []
        for link in soup.find_all('a', href=True):
            href = link.get('href')
            full_url = urljoin(base_url, href)
            if self.is_valid_url(full_url):
                links.append(full_url)
        return links

    def crawl(self, url):

        if len(self.content.keys()) > 300:
            return

        if (url in self.visited_urls):
            return

        if ("https://www.timeout.fr/paris" not in url) or ("/en/" in url) or ('#' in url):
            return

        self.visited_urls.add(url)
        print(f"Crawling: {url}")

        html = self.fetch_page(url)
        if html:
            self.crawl_limit += 1
            links = self.parse_links(html, url)

            # Store content of HTML
            self.content[url] = extract_text(html)
            print("Number of entries:", len(self.content.keys()))
            
            for link in links:
                self.crawl(link)

    def get_content(self):
        return self.content

In [16]:
base_url = "https://www.timeout.fr/paris"  # Replace with the base URL of the website you want to crawl
crawler = WebCrawler(base_url)
crawler.crawl(base_url)

Crawling: https://www.timeout.fr/paris
Number of entries: 1
Crawling: https://www.timeout.fr/paris/search
Number of entries: 2
Crawling: https://www.timeout.fr/paris/actualites
Number of entries: 3
Crawling: https://www.timeout.fr/paris/restaurants-et-bars
Number of entries: 4
Crawling: https://www.timeout.fr/paris/que-faire-a-paris
Number of entries: 5
Crawling: https://www.timeout.fr/paris/culture
Number of entries: 6
Crawling: https://www.timeout.fr/paris/voyage
Number of entries: 7
Crawling: https://www.timeout.fr/paris/hotels
Number of entries: 8
Crawling: https://www.timeout.fr/paris/shopping
Number of entries: 9
Crawling: https://www.timeout.fr/paris/cinema
Number of entries: 10
Crawling: https://www.timeout.fr/paris/theatre
Number of entries: 11
Crawling: https://www.timeout.fr/paris/musique
Number of entries: 12
Crawling: https://www.timeout.fr/paris/art
Number of entries: 13
Crawling: https://www.timeout.fr/paris/en
Number of entries: 14
Crawling: https://www.timeout.fr/paris

In [17]:
documents = crawler.get_content()

### Split documents into chunks

In [18]:
chunks = []
chunks_url_refs = []
chunk_size = 2048
for url_key in list(documents.keys()):
    text = documents[url_key]
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
        chunks_url_refs.append(url_key)

### Create embeddings for each text chunk

In [25]:
import tqdm

In [26]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding

import time
text_embeddings = []
for chunk in tqdm.tqdm(chunks):
    time.sleep(0.1)
    text_embeddings.append(get_text_embedding(chunk))
text_embeddings = np.array(text_embeddings)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1579/1579 [14:55<00:00,  1.76it/s]


In [57]:
embeddings_df = pd.DataFrame(text_embeddings)

In [58]:
import pandas as pd

In [59]:
df = pd.DataFrame({
  "text_chunk": chunks,
  "url_ref_chunk": chunks_url_refs
})

In [60]:
output_df = pd.concat([df, embeddings_df], axis=1)

In [61]:
output_df.to_csv("rag_embeddings.csv", index=False)